### Zaimportowanie odpowiednich bibliotek

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.vectorstores import DocArrayInMemorySearch
import os

### Wczytanie klucza pozwolającego łączenie się z API od OpenAI oraz zawarcie odpowiedniego schematu :
### Dodatkowe informacje do modelu -> zapytanie -> odpowiedź

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()
template = """
        Answer the question based on the context below. If you can't 
        answer the question, reply "I don't know".

        Context: {context}

        Question: {question}
        """
prompt = ChatPromptTemplate.from_template(template) 


### Plik text.txt zawiera  Informacje dla podróżujących do Tajlandii (dla prostoty został zastosowany plik z samą Tajlandią) ze strony 
### [www.gov.pl/web/tajlandia/idp](https://www.gov.pl/web/tajlandia/idp)

### Następnie, te dane są w odpowiedni sposób przekazywane do modelu, dzięki czemu jest w stanie podać precyzyjną odpowiedź

In [ ]:
loader = TextLoader("text.txt", encoding='utf-8')
text_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
document = text_splitter.split_documents(text_documents)
embeddings = OpenAIEmbeddings()

vectorstore2 = DocArrayInMemorySearch.from_documents(document, embeddings)
setup = RunnableParallel(context=vectorstore2.as_retriever(), question=RunnablePassthrough())


In [4]:
chain = setup | prompt | model | parser
anwser = chain.invoke("Czy do podróży po Tajlandii potrzebuje wizy?")
print(anwser)

Nie, obywatele polscy podróżujący do Tajlandii w celu turystycznym do 30 dni nie muszą mieć wizy. Jednak, w przypadku podróży w celu innym niż turystyczny lub gdy pobyt turystyczny ma być dłuższy niż 30 dni, wiza jest wymagana.


![alt text](tajlandia_demo.png)

### Jak widąć, odpowiedź jest identyczna co do treści względem oficjalnej strony gov.pl

### Przykład, jak ten sam model językowy zachowa się na to samo pytanie, bez naszej metody przekazywanie dodatkowych informacji do modelu

In [26]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template(template) 
chain = prompt | model | parser
anwser = model.invoke("Czy do podróży po Tajlandii potrzebuje wizy?")
print(anwser)

content='Odpowiedź zależy od kraju, z którego pochodzisz. Obywatele wielu krajów, w tym Polski, mogą podróżować do Tajlandii bez wizy na pobyt turystyczny do 30 dni. Jednak warto sprawdzić najnowsze informacje na stronie ambasady Tajlandii w Twoim kraju, aby upewnić się, czy potrzebujesz wizy do podróży.' response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 24, 'total_tokens': 132}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e8520f30-3054-4601-b6b9-7429786c8e11-0'


### Odpowiedź jest niekompletna, nie zawiera informacji o możliwości przebywanie bez wizy do 30 dni.

### Demo przedstawia interakcje z dokumentami związanimy ze stroną gov.pl odnośnie informacji dla podróżujących, nastomiast schemat działania będzie podobny dla reszty proponowanych przez nas możliwości związanych z tekstem - zbiernie informacji przez api na temat pogody czy też informacji o ciekawych miejsc do zwiedzenia w zależnosci od wymagań użytkownika

### Nasze rozwiązanie pozwala na komunikowanie się modelu z całą bazą danych, gdzie ilość informacji znacznie przerasta ilość treści jaką model jest w stanie pomieścić - dzięki odpowiedniej selekcji fragmentów danych na podstawie podobieństwa do zapytania, jesteśmy w stanie skorzystać ze złożonych i dużych baz danych do zapewnienia dokładności działania modelu, gdzie zwykłe przekazanie całego fragmentu zakończyło by się błędem i możliwością utraty kluczowych informacji.

## Demo wizja

### Demo poniżej przedstawia zastosowania możliwości wizyjnych i 

In [9]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Czy na tym zdjęciu znajduję się coś co nie mogę wziąć do samolotu?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://www.tsi-mag.com/wp-content/uploads/2023/06/London-City-Airport-Scanners.png",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

![alt text](wizja_demo.png)

In [21]:
response.dict()['choices'][0]['message']['content']

'Na zdjęciu znajduje się plastikowa taca z kilkoma przedmiotami. Przyjrzyjmy się im pod kątem zasad bezpieczeństwa obowiązujących na lotniskach:\n\n1. **Laptop**: Jest dozwolony w bagażu podręcznym.\n2. **Krem ochronny w płynie (SPF 50+)**: Płyny w bagażu podręcznym są dozwolone tylko w pojemnikach do 100 ml. Większe butelki muszą być przewożone w bagażu rejestrowanym.\n3. **Butelka z wodą lub innym napojem**: Płyny powyżej 100 ml nie są dozwolone w bagażu podręcznym. Można zabrać pustą butelkę i napełnić ją po przejściu kontroli bezpieczeństwa.\n4. **Elektronika**: Jeśli są to drobne urządzenia jak zegarek, są one dozwolone w bagażu podręcznym.\n\nPodsumowując: Krem ochronny i butelka z wodą prawdopodobnie przekraczają dozwoloną objętość 100 ml płynów w bagażu podręcznym i powinny zostać przeniesione do bagażu rejestrowanego. Pozostałe przedmioty są dozw'

### Jak widać powyżej, model poprawnie zauważył, że na zdjęciu które może obrazować osobę pakującą się na lot, znajduję się duża butelka wody która może sprawić problemy przy kontroli bezpieczeństwa na lotnisku. Dalsze prace pozwolą na znaczne udoskonalenie analizy zawartości bagażu

'Na zdjęciu znajduje się plastikowa taca z kilkoma przedmiotami. Przyjrzyjmy się im pod kątem zasad bezpieczeństwa obowiązujących na lotniskach:\n\n1. **Laptop**: Jest dozwolony w bagażu podręcznym.\n2. **Krem ochronny w płynie (SPF 50+)**: Płyny w bagażu podręcznym są dozwolone tylko w pojemnikach do 100 ml. Większe butelki muszą być przewożone w bagażu rejestrowanym.\n3. **Butelka z wodą lub innym napojem**: Płyny powyżej 100 ml nie są dozwolone w bagażu podręcznym. Można zabrać pustą bute                                                                                                                            lkę i napełnić ją po przejściu kontroli bezpieczeństwa.\n4. **Elektronika**: Jeśli są to drobne urządzenia jak zegarek, są one dozwolone w bagażu podręcznym.\n\nPodsumowując: Krem ochronny i butelka z wodą prawdopodobnie przekraczają dozwoloną objętość 100 ml płynów w bagażu podręcznym i powinny zostać przeniesione do bagażu rejestrowanego. Pozostałe przedmioty są dozw'



In [25]:
chain = setup | prompt | model | parser
anwser = chain.invoke("Wymień mi najważniejsze informacje o pobycie w Tajlandii")
print(anwser)

- Zarejestrowanie się w systemie Odyseusz przed podróżą zagraniczną
- Kontaktowanie się z Ambasadą RP w Bangkoku w przypadku pytań lub uwag
- Odradzane podróże do południowych prowincji: Pattani, Yala, Narathiwat, Songkhla
- Ostrożność związana z atakami terrorystycznymi w południowych prowincjach
- Ostrzeżenie przed oszustwami związanych z kartami kredytowymi
- Unikanie ofert pracy, które mogą być nielegalne
- Unikanie pobytu na pograniczu z Mjanmą
- Obowiązek meldunkowy i wypełnienie karty wjazdu/wyjazdu
- Konieczność posiadania paszportu w Tajlandii
- Szacunek wobec tradycji religijnych, np. nie dotykanie głowy Tajów
- Uwaga na wymianę walut i korzystanie z taksówek
- Przydatne numery alarmowe: 1669 (pogotowie ratunkowe, straż pożarna, policja), 199 (straż pożarna), 191 (policja), 1155 (policja turystyczna)
- Kontaktowanie się z biurem podróży lub ubezpieczycielem w sytuacjach nagłych
